In [1]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
df = pd.read_csv('/Users/nguyentoan/Desktop/archive (2)/names.csv')
states = pd.read_csv('/Users/nguyentoan/Desktop/archive (2)/states.csv')
df.set_index("Name",inplace = True)
# states.set_index("State",inplace = True)
# data = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")


In [3]:
df.head(10)

,Sex,Count,Year
Name,,,
Emily,F,25735,1997
Jessica,F,21044,1997
Ashley,F,20895,1997
Sarah,F,20712,1997
Hannah,F,20594,1997
Samantha,F,20170,1997
Taylor,F,19502,1997
Alexis,F,17170,1997
Elizabeth,F,15428,1997


In [5]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()
idf1 = states.interactive()
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1880, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year slider', start=1880, step=5, value=1850)

In [8]:
pipeline = (
    idf[
        (idf.Year <= year_slider)
    ]
    .groupby(['Year','Sex'])["Count"].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)
# pipeline

In [9]:
Year_plot = pipeline.hvplot(x = 'Year',by="Sex", y="Count",line_width=2, title="incording to Year")
Year_plot

In [10]:
states.head()


,State,Sex,Year,Name,Count,Total,Count_Normalized
0,OR,F,1910,Dorothy,57,1373,0.041515
1,OR,F,1910,Mary,54,1373,0.039330
2,OR,F,1910,Helen,48,1373,0.034960
3,OR,F,1910,Ruth,46,1373,0.033503
4,OR,F,1910,Margaret,43,1373,0.031318


In [11]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6337734 entries, 0 to 6337733
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   State             object 
 1   Sex               object 
 2   Year              int64  
 3   Name              object 
 4   Count             int64  
 5   Total             int64  
 6   Count_Normalized  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 338.5+ MB


In [27]:
# states["State"].value_counts().to_frame()

In [45]:
# Radio buttons for CO2 measures
year_or= pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=["Count","Total",'Count_Normalized',],
    button_type='success'
)

In [46]:
year_pipeline = (
    idf1[
        (idf1.Year <= year_slider)
    ]
    .groupby(['State', 'Year'])[year_or].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)
# year_pipeline

In [47]:
Yearstate_plot = year_pipeline.hvplot(x = 'Year',by='State',y=year_or,line_width=2, title="incording to Year and state")
Yearstate_plot

In [25]:
Count_table = pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
Count_table1 = year_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
# Count_table
# Count_table1 

In [34]:
count_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.Year == year_slider)
    ]
    .groupby(['Year','Sex'])["Count"].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)
# count_vs_gdp_scatterplot_pipeline
count1_vs_gdp_scatterplot_pipeline = (
    idf1[
        (idf1.Year == year_slider)
    ]
    .groupby(['State', 'Year'])[year_or].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)
# count1_vs_gdp_scatterplot_pipeline

In [37]:
Count_vs_gdp_scatterplot = count_vs_gdp_scatterplot_pipeline.hvplot(x='Year', 
                                                                y='Count', 
                                                                by='Sex', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
# Count_vs_gdp_scatterplot
Count1_vs_gdp_scatterplot = count1_vs_gdp_scatterplot_pipeline.hvplot(x='Year', 
                                                                y=year_or, 
                                                                by='State', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
Count1_vs_gdp_scatterplot

In [50]:
year_or= pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=["Count","Total",'Count_Normalized',],
    button_type='success'
)
# count_name = pn.widgets.RadioButtonGroup(
#     name='Y axis', 
#     options=["Count","Total",'Count_Normalized',], 
#     button_type='success'
# )

continents_excl_world = ['SC', 'SD', 'TN', 'TX', 'UT', 'VA']

bar_pipeline = (
    idf1[
        (idf1.Year == year_slider)&
        (idf1.State.isin(continents_excl_world))
    ]
    .groupby(['State', 'Year'])[year_or].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

In [52]:
bar_plot = bar_pipeline.hvplot(kind='bar', 
                                x='State', 
                                y=year_or, 
                                title='US Baby Name Popularity')
bar_plot

In [58]:
# #Layout using Template
# template = pn.template.FastListTemplate(
#     title='US Baby Name Popularity dashboard', 
#     sidebar=[pn.pane.Markdown("# US Baby Name Popularity and Climate Change"), 
#              pn.pane.Markdown("#### US Baby Name Popularity count"), 
#              pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
#              pn.pane.Markdown("## Settings"),   
#              year_slider],
#     main=[pn.Row(pn.Column(yaxis_or, 
#                            Yearstate_plot.panel(width=700), margin=(0,25)), 
#                  Count_table1.panel(width=500)), 
#           pn.Row(pn.Column(Count1_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
#                  pn.Column(year_or,bar_plot.panel(width=600)))],
#     accent_base_color="#88d8b0",
#     header_background="#88d8b0",
# )
# template.show()
# template.servable();